<a href="https://colab.research.google.com/github/jirslad/project_classification/blob/main/demo_training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0a. Setup git repo

In [ ]:
# USE_DRIVE = False: Fully use local memory. (long downloading time of a dataset)
# USE_DRIVE = True: Use local memory, only use Drive to copy Food101 dataset archive to local memory.
USE_DRIVE = True

if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/gdrive/') # , force_remount=True

In [ ]:
# clone repo to local memory
%cd /content/
!git clone https://github.com/jirslad/project_classification.git
%cd /content/project_classification/
!git pull

##0b. Install additional requirements

In [ ]:
# the newest version of mlxtend might require Colab runtime restart
try:
  import torchinfo, torchmetrics, mlxtend
except:
  %pip install torchinfo torchmetrics mlxtend==0.21
  import torchinfo, torchmetrics, mlxtend

# 1. Download and process dataset
A) Food-101: Download the whole dataset do Drive once. Since then, copy if from Drive to local memory.

B) Food-101 subset of 3 classes: Extract only several classes (pizza, steak, sushi). Script easily modifiable for arbitrary classes.

C) DTD:
Download dataset to Drive. Process it to create `class_names.txt` with ordered class names and `annotations.csv` with image paths and corresponding labels (as class indexes, e.g. `'3 18 42'`).

In [ ]:
# # ONLY ONCE: Download Food101 dataset archive to Drive
# %mkdir /content/gdrive/MyDrive/computer_vision/project_classification/datasets/
# %cd /content/gdrive/MyDrive/computer_vision/project_classification/datasets/
# import urllib.request
# urllib.request.urlretrieve('https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz', 'food-101.tar.gz')

In [ ]:
# A) for Food101 dataset
if USE_DRIVE:
  %mkdir /content/project_classification/datasets/
  %cp -av /content/gdrive/MyDrive/computer_vision/project_classification/datasets/food-101.tar.gz /content/project_classification/datasets/food-101.tar.gz
  import tarfile
  with tarfile.open("/content/project_classification/datasets/food-101.tar.gz", "r") as f:
    f.extractall("/content/project_classification/datasets/")

  # B) for whole pizza_steak_sushi subset (1000 imgs per class)
  !python prepare_dataset.py --datasets-dir datasets --dataset FoodSubset

  # for pizza_steak_sushi custom downloaded images
  %mkdir -p /content/project_classification/datasets/pizza_steak_sushi
  %cp -r /content/gdrive/MyDrive/computer_vision/project_classification/datasets/pizza_steak_sushi/custom /content/project_classification/datasets/pizza_steak_sushi
  
# C) for DTD dataset
else:
  !python prepare_dataset.py --datasets-dir datasets --dataset DTD

# 2. Train a model

In [ ]:
%run train.py --data-path datasets/pizza_steak_sushi --model efficientnetB0 \
              --epochs 10 --lr 5e-4 --batch 32 --split-ratio 1 1 \
              --track --plot

In [ ]:
# Show experiments in TensorBoard
%reload_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# download "runs" folder with experiment results
from google.colab import files
!zip -r /content/runs.zip /content/project_classification/runs
files.download("/content/runs.zip")

In [ ]:
# download trained models to Drive
# %cp -av /content/project_classification/models /content/gdrive/MyDrive/computer_vision/project_classification

# copy a trained model to local memory
# %mkdir /content/project_classification/models
# %cp -av /content/gdrive/MyDrive/computer_vision/project_classification/models/vitB16_100perc-data_4ep_0.000015lr.pt /content/project_classification/models/

## 3. Run inference

In [ ]:
%run inference.py --imgs-path datasets/pizza_steak_sushi/custom --model-path models/vitB16_100perc-data_4ep_0.000015lr.pt \
                  --rows 5 --columns 9 --output-classes 3